In [3]:
!pip3 install yfinance requests beautifulsoup4 boto3

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 73 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 139 kB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 505 kB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 8.1 MB 6.6 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 7.2 MB/s eta 0:00:01
     |████████████████████████████████| 11.3 MB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 5.2 MB 9.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 11.4 MB/s eta 0:00:01
  distutils: /private/var/folders/0b/hbkb4qy916350sky43xyx5tm0000gr/T/pip-build-env-juy5lqbq/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/0b/hbkb4qy916350sky43xyx5tm0000gr/T/pip-build-env-juy5lqbq/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = False
  home = No

In [8]:
!pip3 install yfinance requests beautifulsoup4 boto3 --upgrade

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [9]:
import yfinance as yf
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import ast
import json
import boto3

# AWS Bedrock の設定
BEDROCK_MODEL_ID = 'anthropic.claude-3-sonnet-20240229-v1:0'
BEDROCK_REGION = 'us-east-1'

def get_article_text(url):
    """
    指定された URL のウェブページからテキストを抽出する
    """
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        article_text = ' '.join([p.get_text() for p in soup.find_all('p')])
        return article_text
    except Exception as e:
        print(f"Error retrieving article text: {e}")
        return ""

def get_stock_data(ticker, years):
    """
    指定した銘柄の株価データ、財務諸表、ニュースを取得する
    """
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=years*365)

    # Yahoo! Financeからクッキーとクラムを取得
    url = "https://finance.yahoo.com/quote/%s?p=%s" % (ticker, ticker)
    session = requests.Session()
    response = session.get(url)
    cookie = response.cookies.get_dict()
    crumb = None
    for line in response.text.splitlines():
        if line.startswith('CrumbStore'):
            crumb = line.split(':')[2].strip('" }')
            break

    # yf.Tickerオブジェクトにクッキーとクラムを設定
    stock = yf.Ticker(ticker, session=session)
    if crumb:
        stock.crumb = crumb
    stock.cookies = cookie

    # 株価データを取得
    hist_data = stock.history(start=start_date, end=end_date)

    # 財務諸表を取得
    try:
        balance_sheet = stock.balance_sheet
        financials = stock.financials
    except:
        balance_sheet = None
        financials = None

    # ニュース記事を取得
    news = stock.news

    return hist_data, balance_sheet, financials, news

def get_comparable_companies(ticker, hist_data, balance_sheet, financials, news):
    """
    指定した銘柄の類似企業のリストを取得する
    """
    system_prompt = f"You are a financial analyst assistant. Analyze the given data for {ticker} and suggest a few comparable companies to consider. Do so in a Python-parseable list."

    news_text = "\n\n".join([f"---\n\nTitle: {article['title']}\nText: {get_article_text(article['link'])}" for article in news])

    messages = [
        {"role": "user", "content": f"Historical price data:\n{hist_data.tail().to_string()}\n\nBalance Sheet:\n{balance_sheet.to_string()}\n\nFinancial Statements:\n{financials.to_string()}\n\nNews articles:\n{news_text}\n\n----\n\nNow, suggest a few comparable companies to consider, in a Python-parseable list. Return nothing but the list. Make sure the companies are in the form of their tickers."},
    ]

    response = invoke_bedrock_model(
        system_prompt=system_prompt,
        messages=messages,
        max_tokens=2000,
        temperature=0.5
    )

    if response is None:
        return []
    else:
        return ast.literal_eval(response)

def compare_companies(main_ticker, main_data, comp_ticker, comp_data):
    """
    2つの企業のデータを比較し、詳細な分析を提供する
    """
    system_prompt = f"You are a financial analyst assistant. Compare the data of {main_ticker} against {comp_ticker} and provide a detailed comparison, like a world-class analyst would. Be measured and discerning. Truly think about the positives and negatives of each company. Be sure of your analysis. You are a skeptical investor."

    messages = [
        {"role": "user", "content": f"Data for {main_ticker}:\n\nHistorical price data:\n{main_data['hist_data'].tail().to_string()}\n\nBalance Sheet:\n{main_data['balance_sheet'].to_string()}\n\nFinancial Statements:\n{main_data['financials'].to_string()}\n\n----\n\nData for {comp_ticker}:\n\nHistorical price data:\n{comp_data['hist_data'].tail().to_string()}\n\nBalance Sheet:\n{comp_data['balance_sheet'].to_string()}\n\nFinancial Statements:\n{comp_data['financials'].to_string()}\n\n----\n\nNow, provide a detailed comparison of {main_ticker} against {comp_ticker}. Explain your thinking very clearly."},
    ]

    response = invoke_bedrock_model(
        system_prompt=system_prompt,
        messages=messages,
        max_tokens=3000,
        temperature=0.5
    )

    return response

def get_sentiment_analysis(ticker, news):
    """
    指定した銘柄のニュース記事の感情分析を行う
    """
    system_prompt = f"You are a sentiment analysis assistant. Analyze the sentiment of the given news articles for {ticker} and provide a summary of the overall sentiment and any notable changes over time. Be measured and discerning. You are a skeptical investor."

    news_text = "\n\n".join([f"---\n\nDate: {datetime.fromtimestamp(article['providerPublishTime']).strftime('%Y-%m-%d')}\nTitle: {article['title']}\nText: {get_article_text(article['link'])}" for article in news])

    messages = [
        {"role": "user", "content": f"News articles for {ticker}:\n{news_text}\n\n----\n\nProvide a summary of the overall sentiment and any notable changes over time."},
    ]

    response = invoke_bedrock_model(
        system_prompt=system_prompt,
        messages=messages,
        max_tokens=2000,
        temperature=0.5
    )

    return response

def get_analyst_ratings(ticker):
    """
    指定した銘柄のアナリスト評価を取得する
    """
    stock = yf.Ticker(ticker)
    recommendations = stock.recommendations
    if recommendations is None or recommendations.empty:
        return "No analyst ratings available."

    latest_rating = recommendations.iloc[-1]

    firm = latest_rating.get('Firm', 'N/A')
    to_grade = latest_rating.get('To Grade', 'N/A')
    action = latest_rating.get('Action', 'N/A')

    rating_summary = f"Latest analyst rating for {ticker}:\nFirm: {firm}\nTo Grade: {to_grade}\nAction: {action}"

    return rating_summary

def get_industry_analysis(ticker):
    """
    指定した銘柄の業界分析を行う
    """
    stock = yf.Ticker(ticker)
    industry = stock.info['industry']
    sector = stock.info['sector']

    system_prompt = f"You are an industry analysis assistant. Provide an analysis of the {industry} industry and {sector} sector, including trends, growth prospects, regulatory changes, and competitive landscape. Be measured and discerning. Truly think about the positives and negatives of the stock. Be sure of your analysis. You are a skeptical investor."

    messages = [
        {"role": "user", "content": f"Provide an analysis of the {industry} industry and {sector} sector."},
    ]

    response = invoke_bedrock_model(
        system_prompt=system_prompt,
        messages=messages,
        max_tokens=2000,
        temperature=0.5
    )

    return response

def get_final_analysis(ticker, comparisons, sentiment_analysis, analyst_ratings, industry_analysis):
    """
    各種分析結果を統合し、最終的な投資勧告を生成する
    """
    system_prompt = f"You are a financial analyst providing a final investment recommendation for {ticker} based on the given data and analyses. Be measured and discerning. Truly think about the positives and negatives of the stock. Be sure of your analysis. You are a skeptical investor."

    messages = [
        {"role": "user", "content": f"Ticker: {ticker}\n\nComparative Analysis:\n{json.dumps(comparisons, indent=2)}\n\nSentiment Analysis:\n{sentiment_analysis}\n\nAnalyst Ratings:\n{analyst_ratings}\n\nIndustry Analysis:\n{industry_analysis}\n\nBased on the provided data and analyses, please provide a comprehensive investment analysis and recommendation for {ticker}. Consider the company's financial strength, growth prospects, competitive position, and potential risks. Provide a clear and concise recommendation on whether to buy, hold, or sell the stock, along with supporting rationale."},
    ]

    response = invoke_bedrock_model(
        system_prompt=system_prompt,
        messages=messages,
        max_tokens=3000,
        temperature=0.5
    )

    return response

def generate_ticker_ideas(industry):
    """
    指定された業界の主要企業の銘柄リストを生成する
    """
    system_prompt = f"You are a financial analyst assistant. Generate a list of 5 ticker symbols for major companies in the {industry} industry, as a Python-parseable list."

    messages = [
        {"role": "user", "content": f"Please provide a list of 5 ticker symbols for major companies in the {industry} industry as a Python-parseable list. Only respond with the list, no other text."},
    ]

    response = invoke_bedrock_model(
        system_prompt=system_prompt,
        messages=messages,
        max_tokens=200,
        temperature=0.5
    )

    if response is None:
        return []
    else:
        ticker_list = ast.literal_eval(response)
        return [ticker.strip() for ticker in ticker_list]

def get_current_price(ticker):
    """
    指定した銘柄の現在の株価を取得する
    """
    stock = yf.Ticker(ticker)
    data = stock.history(period='1d', interval='1m')
    return data['Close'][-1]

def rank_companies(industry, analyses, prices):
    """
    各社の分析結果と現在の株価を基に、投資価値の順位付けを行う
    """
    system_prompt = f"You are a financial analyst providing a ranking of companies in the {industry} industry based on their investment potential. Be discerning and sharp. Truly think about whether a stock is valuable or not. You are a skeptical investor."

    analysis_text = "\n\n".join(
        f"Ticker: {ticker}\nCurrent Price: {prices.get(ticker, 'N/A')}\nAnalysis:\n{analysis}"
        for ticker, analysis in analyses.items()
    )

    messages = [
        {"role": "user", "content": f"Industry: {industry}\n\nCompany Analyses:\n{analysis_text}\n\nBased on the provided analyses, please rank the companies from most attractive to least attractive for investment. Provide a brief rationale for your ranking. In each rationale, include the current price (if available) and a price target."},
    ]

    response = invoke_bedrock_model(
        system_prompt=system_prompt,
        messages=messages,
        max_tokens=3000,
        temperature=0.5
    )

    return response


def invoke_bedrock_model(system_prompt, messages, max_tokens, temperature):
    """
    Amazon Bedrock にリクエストを送信し、レスポンスを取得する
    """
    try:
        bedrock_runtime = boto3.client('bedrock-runtime', region_name=BEDROCK_REGION)

        request_body = json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "temperature": temperature,
            "messages": messages,
            "system": system_prompt
        })

        response = bedrock_runtime.invoke_model(
            modelId=BEDROCK_MODEL_ID,
            contentType='application/json',
            accept='application/json',
            body=request_body
        )

        response_body = json.loads(response['body'].read())
        response_text = response_body['content'][0]['text']

        return response_text
    except Exception as e:
        print(f"Error invoking Bedrock model: {e}")
        return None

# ユーザー入力
industry = input("Enter the industry to analyze: ")
years = int(input("Enter the number of years for analysis: "))

# 業界の主要企業の銘柄リストを生成
tickers = generate_ticker_ideas(industry)
print(f"\nTicker Ideas for {industry} Industry:")
print(", ".join(tickers))

# 各社の分析を実行
analyses = {}
prices = {}
for ticker in tickers:
    try:
        print(f"\nAnalyzing {ticker}...")
        hist_data, balance_sheet, financials, news = get_stock_data(ticker, years)
        main_data = {
            'hist_data': hist_data,
            'balance_sheet': balance_sheet,
            'financials': financials,
            'news': news
        }
        comparable_companies = get_comparable_companies(ticker, hist_data, balance_sheet, financials, news)
        sentiment_analysis = get_sentiment_analysis(ticker, news)
        analyst_ratings = get_analyst_ratings(ticker)
        industry_analysis = get_industry_analysis(ticker)
        comparisons = {}
        for comp_ticker in comparable_companies:
            comp_data = get_stock_data(comp_ticker, years)
            comparisons[comp_ticker] = compare_companies(ticker, main_data, comp_ticker, comp_data)
        final_analysis = get_final_analysis(ticker, comparisons, sentiment_analysis, analyst_ratings, industry_analysis)
        analyses[ticker] = final_analysis
        prices[ticker] = get_current_price(ticker)
    except Exception as e:
        print(f"Error analyzing {ticker}: {e}")

# 企業の順位付け
ranking = rank_companies(industry, analyses, prices)
print(f"\nRanking of Companies in the {industry} Industry:")
print(ranking)